In [108]:
import pandas as pd
import glob
import plotly.graph_objects as go
import numpy as np


# %%
files = glob.glob('../results_data/multiple_seed/*')
labels = ['pix2pix', 'pix2pix L1', 'pix2pix L1+reset', 'pix2pix patches', 'cycleGAN', 'cycleGAN patches']
file_names = ['test_pix2pix_PAS2IF_van_patches', 'test_pix2pix_PAS2IF_L1_patches', 'test_pix2pix_PAS2IF_L1_reset_patches', '', 'test_cycle_PAS2IF_van_patches', '']
metrics = ['SSIM', 'MAE', 'MSE', 'FID', 'Dice podo. object', 'Dice podo. pixel', 'Dice glom. pixel']

In [87]:
# Read Multi Seeds
df_final = pd.DataFrame()
for file in files:
    if 'Hamburg' in file:
        df_temp = pd.read_excel(file)
        df_temp = df_temp[['Network dice object', 'Network dice pixel']]
        df = pd.DataFrame()
        cell_type = 'glom.' if 'glom' in file else 'podo.'
        df['value'] = df_temp['Network dice object'].append(df_temp['Network dice pixel'])
        df['name'] = 'pix2pix'
        df['metric'] = np.concatenate([np.repeat(f'Dice {cell_type} object',len(df_temp)),
                           np.repeat(f'Dice {cell_type} pixel', len(df_temp))])
    else:
        name = file.split('/')[3].split(' ')[0]
        metric = file.split(' ')[1].split('.')[0]
        df = pd.read_csv(file)
        df = df.filter(regex=r' (?!.*?(MIN|MAX)).*', axis=1)
        df['value'] = df.mean(axis=1)
        df['metric'] = metric
        df['name'] = name
    df_final = pd.concat([df_final, df[['name', 'metric', 'value']]])


asdf
asdf


In [91]:
for label, file_name in zip(labels, file_names):
    df_final = df_final.replace({file_name: label})

In [109]:
def boxplot(df: pd.DataFrame, metric: str, labels:list, export=False) -> None:
    df_plot = df[df.metric == metric]
    colors = ['#4038E8', '#3A63F2', '#408ADB', '#3AC4F2', '#30F541', '#3EF595']
    fig = go.Figure()
    for col, name in zip(colors, labels):
        fig.add_trace(go.Box(y=df_plot['value'][df_plot.name == name],
            name=name, marker_color=col, boxmean=True, showlegend=False, boxpoints='all'
        ))
    fig.update_layout(template='simple_white', yaxis_title=metric, font_family="Computer Modern")
    fig.update_xaxes(categoryorder='array', categoryarray=labels)
    fig.show()

    if export:
        fig.write_image(f'../graphics/{metric}.png')

for metric in metrics:
    boxplot(df_final, metric, labels=labels, export=True)

